In [40]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [41]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res.json")):
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        with open(os.path.join(sub_folder_path, "result.json")) as f:
            last_text = f.readlines()[-1]
            epochs_trained = re.search('epoch": [0-9]*', last_text).group(0).split(": ")[-1]
        params["folder"] = folder
        params["epochs_trained"] = int(epochs_trained)
        trials.append((params, eval_res))
open_trials = [folder for folder in os.listdir(ray_folder) if folder.startswith("hyperopt")]
trials_run_folders = [f[0]["folder"] for f in trials]
open_trials = [f for f in open_trials if f not in trials_run_folders]
print(f"Trials run: {len(trials)} Trials still open: {len(open_trials)}")
with open("still_open_qm9_full_full_mat.txt", "w") as f: 
    f.write("\n".join(open_trials))

Trials run: 15 Trials still open: 85


In [42]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
print(f"Found {len(trials)} trials in {ray_folder}")
for trial in trials: 
    print(trial[0]["folder"])
    print(trial[1]["cummulative_stats"])
    print(trial[0]["epochs_trained"])
    print("---")


Found 15 trials in /home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss.py
hyperopt_train_036e6_00001_1_batch_size=16,data_aug_factor=2,edge_threshold_val=3.5698,hidden_dim=128,lr=0.0009,message_net_dropou_2025-07-18_09-31-42
{'energy_abs': {'mean': -1086.2075915618307, 'std': 271.67350948402606}, 'iterations': {'mean': 12.22, 'std': 1.6886681142249353}, 'energy_rel': {'mean': -0.6920868749607015, 'std': 0.03634697261424198}, 'diis': {'mean': 0.20714547915559142, 'std': 0.02992177985057091}, 'rmse': {'mean': 0.0094223677499698, 'std': 0.0028106254813412036}}
5
---
hyperopt_train_036e6_00050_50_batch_size=32,data_aug_factor=3,edge_threshold_val=2.6735,hidden_dim=256,lr=0.0024,message_net_dropo_2025-07-18_13-04-00
{'energy_abs': {'mean': -1079.3584894043072, 'std': 271.7995578227922}, 'iterations': {'mean': 12.32, 'std': 2.043917806566595}, 'energy_rel': {'mean': -0.6874294921790205, 'std': 0.0370295438930694}, 'diis': 

In [43]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.3f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 12.2(1.7) & -1086.2(271.7) & -0.692(0.036) & 0.207(0.030) & 0.0094(0.0028) \\
1 & 12.3(2.0) & -1079.4(271.8) & -0.687(0.037) & 0.204(0.029) & 0.0095(0.0029) \\
2 & 12.4(2.0) & -1041.4(258.6) & -0.664(0.032) & 0.190(0.029) & 0.0092(0.0026) \\
3 & 12.4(2.3) & -1062.1(268.8) & -0.676(0.038) & 0.213(0.067) & 0.0099(0.0040) \\
4 & 12.4(1.4) & -1113.2(281.5) & -0.709(0.041) & 0.219(0.031) & 0.0094(0.0023) \\
5 & 12.5(2.6) & -1091.1(278.3) & -0.694(0.041) & 0.206(0.026) & 0.0086(0.0023) \\
6 & 12.7(3.1) & -1082.0(274.2) & -0.689(0.039) & 0.209(0.035) & 0.0099(0.0034) \\
7 & 12.7(2.1) & -1095.0(277.7) & -0.697(0.042) & 0.262(0.331) & 0.0101(0.0033) \\
8 & 13.0(3.1) & -1083.3(277.5) & -0.689(0.044) & 0.212(0.035) & 0.0105(0.0043) \\
9 & 13.0(2.7) & -1071.9(281.1) & -0.681(0.050) & 0.312(0.772) & 0.0102(0.0036) \\
\bottomrule
\end{tabular}



In [44]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 128 & 256 & 256 & 128 & 128 & 128 & 512 & 128 & 256 & 512 \\
Batch Size & 16 & 32 & 8 & 8 & 8 & 8 & 32 & 32 & 32 & 32 \\
Data aug. & 2.00 & 3.00 & 1.00 & 1.00 & 1.00 & 4.00 & 2.00 & 4.00 & 4.00 & 4.00 \\
Edge thres. & 3.57 & 2.67 & 2.99 & 3.34 & 2.12 & 2.51 & 2.46 & 3.95 & 3.13 & 3.96 \\
Message p. steps & 3 & 2 & 4 & 3 & 3 & 5 & 5 & 3 & 3 & 3 \\
Message Net Dropout & 0.27 & 0.20 & 0.02 & 0.27 & 0.18 & 0.25 & 0.07 & 0.20 & 0.08 & 0.01 \\
Message Net Layers & 3 & 2 & 4 & 2 & 4 & 4 & 2 & 4 & 3 & 3 \\
Learn Rate & 9.04e-04 & 2.42e-03 & 1.27e-04 & 1.47e-04 & 2.64e-04 & 2.36e-04 & 1.32e-04 & 5.89e-04 & 1.60e-04 & 1.22e-04 \\
Weight Decay & 2.13e-05 & 4.19e-05 & 1.45e-05 & 5.31e-05 & 1.98e-05 & 5.91e-04 & 1.18e-04 & 3.70e-04 & 6.24e-05 & 7.85e-05 \\
\bottomrule
\end{tabular}

